# Load the data

In [ ]:
## setup analysis
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.lib.purification import purify_pq as ppq; reload(ppq)
from analysis.lib.purification import purify_analysis as pa; reload(pa)
%matplotlib inline

folder_lt3 = r'D:\measuring\data\Purification_lt3_raw_data'
folder_lt4 = r'D:\measuring\data\Purification_lt4_raw_data'
# addendum = r'\Purify_XX_First_Attempts'
# addendum = r'\Purify_25percent_theta'
# addendum = r'\Purify_35percent_theta'
addendum = r'\Purify_15percent_theta'
folder_lt3 = folder_lt3+addendum
folder_lt4 = folder_lt4+addendum
### ssro calibrations
day = '20160716_' # for ssro calibration

ssro_calib_lt3 = day+'091002'
ssro_calib_lt4 = day+'113952'

Purify_analysis = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)

In [ ]:

### list of good timestamps and RO calibration
all_lt3,all_lt4 = [],[]

### get all valid time stamps. remember that data has to be stored locally.
# msmt_days_pi25_XX_ZZ  = ['20160808','20160807','20160722','20160721','20160720']
# msmt_days_pi25_YY = ['20160808','20160807','20160722','20160724','20160725'] 
# msmt_days_pi25_XX_YY_ZZ_new_timings  = ['20160811']
# msmt_days_old = msmt_days_pi25_XX_ZZ
# msmt_days = msmt_days_pi25_XX_YY_ZZ_new_timings

# msmt_days_pi2 = ['20160714','20160715','20160716','20160718','20160719']
# msmt_days_pi2_YY = ['20160726','20160727']
# msmt_days_pi2_new_timings = ['20160816']
# msmt_days_old = msmt_days_pi2
# msmt_days = msmt_days_pi2_new_timings

# msmt_days_pi35 = ['20160818','20160819']
# msmt_days_old = []
# msmt_days = msmt_days_pi35


msmt_days_pi15 = ['20160816','20160820','20160821','20160822']
msmt_days_old = []
msmt_days = msmt_days_pi15

bases = 'YY'

offsets = [] # Hold offset to compensate for timing change for new APD
correction_time = 2772.5e3-2791.5e3
offsets_ch1 = []
start_offset_ch1 = 1.8e3


for d in msmt_days_old:
    tstamp_lt3,tstamp_lt4 = Purify_analysis.tstamps_for_both_setups(d,contains = bases)#,newest_tstamp = '110000') ### newest timestamp allows for only taking parts of a day.
    all_lt3.extend(tstamp_lt3)
    all_lt4.extend(tstamp_lt4)
    offsets.extend(np.zeros(np.shape(tstamp_lt3)))
    offsets_ch1.extend(np.zeros(np.shape(tstamp_lt4)))

for d in msmt_days:

    tstamp_lt3,tstamp_lt4 = Purify_analysis.tstamps_for_both_setups(d,contains = bases)#,newest_tstamp = '110000') ### newest timestamp allows for only taking parts of a day.
    all_lt3.extend(tstamp_lt3)
    all_lt4.extend(tstamp_lt4)
    offsets.extend(np.zeros(np.shape(tstamp_lt3))+correction_time)
    offsets_ch1.extend(np.zeros(np.shape(tstamp_lt4))+start_offset_ch1)

# print all_lt3
# print all_lt4


In [ ]:
Purify_analysis.load_raw_data(all_lt3,all_lt4,verbose = True)
Purify_analysis.correct_pq_times(offsets = offsets, offsets_ch1 = offsets_ch1)

In [ ]:
Purify_analysis.check_tail_w1_w2(st_start = 2773e3,st_len = 40e3) #### tail counts
Purify_analysis.check_tail_w1_w2(st_start = 2769e3,st_len = 6e3)#### pulse counts

In [ ]:
### autocorrelation of photon clicks --> do we have huge waiting times in the experiment
# reload(pq_plots)
# a_lt4 = ppq.purifyPQAnalysis(tb.latest_data(tstamp_lt4[0],folder = folder_lt4))
# print a_lt4
# pq_plots.plot_autocorrelation_histogram(a_lt4.pqf,start = 0,length = 50e3,binsize=0.1e3)

### Filtering and correlations

In [ ]:
photon_channel = 2 # 0 or 1; 2 means both HH detectors
st_start = 2772.5e3#2763e3
st_len       = 40e3 #50 ns
st_len_w2    = 40e3
ch1_offset = -start_offset_ch1
max_reps_w2 = 150

In [ ]:
### this window can be used to analyze one of the full timetrace in order to get a feeling for the tail and the windows.
### later on all timestamps should be combined to get a clear pikkcture

a_lt4 = ppq.purifyPQAnalysis(tb.latest_data(all_lt4[1],folder = folder_lt4))
vert0 = st_start/1000. ### start of the first window

# BK_analysis.plot_total_histogram(1,st_start-20e3,200e3,1e3,log_plot=True)
pq_plots.plot_marker_filter_comparison(a_lt4.pqf,mrkr_chan = 1,start =2767e3,  length = 50e3, hist_binsize = 0.1e3, save = False, log=True,plot_threshold_ch0 =vert0,plot_threshold_ch1 =vert0+ch1_offset*1e-3) #start =7480

# pq_plots.plot_marker_filter_comparison(a_lt4.pqf,mrkr_chan = 1,start =1965e3+1000e3,  length = 10e3, hist_binsize = 0.1e3, save = False, log=False,plot_threshold_ch0 =vert1,plot_threshold_ch1 =vert1) #start =7480
# pq_plots.plot_photon_hist(b.pqf,start =1965+500,  length = 250, hist_binsize = 1, save = False, log=True) #start =7480

In [ ]:
### return the length of events without temporal filter:

no_of_events = 0
for i in range(len(Purify_analysis.lt3_dict['ssro_results'])):
    no_of_events += len(Purify_analysis.lt3_dict['ssro_results'][i])

print 'number of unfiltered (temporal and CR) events', no_of_events



### apply temporal filter (see above for filters)
#Purify_analysis.filter_CR_after(threshold = 1,verbose = False) ## NOT IMPLEMENTED
Purify_analysis.apply_temporal_filters_to_prefiltered_data(st_start = st_start,
                                                       st_len = st_len,
                                                       st_len_w2 = st_len_w2
                                                       )
# ch1_offset = ch1_offset NOT IMPLEMENTED

In [ ]:
Purify_analysis.apply_sync_filter_w1_w2(verbose = True,max_w2 = max_reps_w2)
Purify_analysis.apply_is_purified_filter(signature = '11', verbose = True)
Purify_analysis.apply_CR_before_filter(verbose=False)
Purify_analysis.apply_CR_after_filter(verbose=False)
Purify_analysis.attach_state_filtered_syncs(verbose = False)

In [ ]:
### applies the time fitlered sync numbers to the adwin RO results of each setup

Purify_analysis.correlate_RO_results(verbose = True,apply_ROC = True)

In [ ]:
### correlations vs. window start?
Purify_analysis.sweep_filter_parameter_vs_correlations('st_start',np.linspace(2766e3,2779e3,20),apply_ROC = True)

In [ ]:
Purify_analysis.sweep_filter_parameter_vs_correlations('st_len',np.linspace(10e3,50e3,30),apply_ROC = True)

In [ ]:
Purify_analysis.sweep_filter_parameter_vs_correlations('max_reps_w2',np.arange(20,500,30),apply_ROC = True)

In [ ]:
Purify_analysis.sweep_filter_parameter_vs_correlations('max_reps_w1',np.arange(50,1000,50),apply_ROC = True)

In [ ]:
Purify_analysis.sweep_filter_parameter_vs_correlations('min_cr_lt4_after',np.arange(0,20,2),apply_ROC = True)

In [ ]:
Purify_analysis.sweep_filter_parameter_vs_correlations('min_cr_lt4_before',np.arange(25,33,2),apply_ROC = True)

In [ ]:
Purify_analysis.sweep_filter_parameter_vs_correlations('min_cr_lt3_after',np.arange(0,20,2),apply_ROC = True)

In [ ]:
Purify_analysis.sweep_filter_parameter_vs_correlations('min_cr_lt3_before',np.arange(20,34,2),apply_ROC = True)

In [ ]:
def format_plot(fig,ax):
    linewidths = 1.5
    textsize = 14
    ax.tick_params(labelsize = textsize,width =linewidths)
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(linewidths)
    ax.yaxis.label.set_size(textsize)
    ax.xaxis.label.set_size(textsize)
    
    
def num2str(num, precision): 
    return "%0.*f" % (precision, num)

save_plot_folder = r'D:\measuring\data\purification_plots'
def save_and_close_plot(f = save_plot_folder, save = False, name = None):

    if name == None:
        name = 'Results'

    if save:
        plt.savefig(os.path.join(f,name+'.pdf'),format='pdf',bbox_inches='tight')
        plt.savefig(os.path.join(f,name+'.png'),format='png',bbox_inches='tight')

    plt.show()
    plt.close('all')
    
def plot_parity_results(corr,err=np.array([0,0]),events = np.array([0,0]),state = 'parity_mixed'):


    fig = plt.figure()
    ax = plt.subplot()
    plt.ylabel('Expectation value')
    # plt.title('Corrected')
    plt.bar(np.arange(0,3), np.squeeze(np.asarray(corr)), yerr=err, align = 'center',
            color = '#436CDE',linewidth = 1.5,error_kw=dict(lw = 1.5,capthick=1.5))
    for k in range(3):
        plt.text(k,0.02, num2str(np.squeeze(np.asarray(corr))[k],2), ha = 'center')
#         plt.text(k,err[k]+corr[k]+0.02, num2str(np.squeeze(np.asarray(events))[k],0), ha = 'center')


    plt.ylim([-0.7,0.7])
    plt.xticks([0,1,2], ['<XX>','<YY>','<ZZ>'])
    ax.set_xlim(-0.6,2.6)
    format_plot(fig,ax)
    plt.axhline(0,lw=1.5,color = 'black')
    ax.tick_params(top = 'off',bottom = 'off')
    save_and_close_plot(save = True,name = 'Entangled_state_minus')
    


In [ ]:
### XX probabilities values:
XX_p = [ 0.02194542,  0.44107568,  0.33391541,  0.20306349]
XX_p_u = [ 0.01780569,  0.0419028,   0.03794178,  0.03850672]

### YY probabilities values:
YY_p = [ 0.11730046,  0.32593922,  0.3906194,   0.16614093]
YY_p_u = [ 0.0338351,   0.04349794,  0.04302538,  0.03916723]

### ZZ probabiliteis values:
ZZ_p = [ 0.33517993,  0.20705804,  0.08719132,  0.37057072]
ZZ_p_u = [ 0.04734693,  0.04289617,  0.03567978,  0.04477041]


### calculate expectation values
YY = (YY_p[0]+YY_p[3]-0.5)*2
YY_u = np.sqrt(YY_p_u[0]**2+YY_p_u[3]**2)*2
ZZ = (ZZ_p[0]+ZZ_p[3]-0.5)*2
ZZ_u = np.sqrt(ZZ_p_u[0]**2+ZZ_p_u[3]**2)*2
XX = (XX_p[0]+XX_p[3]-0.5)*2
XX_u = np.sqrt(XX_p_u[0]**2+XX_p_u[3]**2)*2

plot_parity_results(np.array([XX,YY,ZZ]),err = np.array([XX_u,YY_u,ZZ_u]))